In [5]:
import pandas as pd 
import matplotlib.pyplot as plt 
import numpy as np

In [6]:
date_cols = [
    'Auftragseingang', 
    'Auftragsende_SOLL',
    'AFO_Start_SOLL',
    'AFO_Ende_SOLL',
    'AFO_Start_IST',
    'AFO_Ende_IST'
]

data = pd.read_csv(
    '../data/processed/data_cleaned_timestamp_1.csv',
    parse_dates=date_cols,
    sep=',',
    low_memory=False
)

In [12]:
data.head()

,AuftragsID,BauteilID,Bauteilbezeichnung,Auftragseingang,Priorität,Auftragsende_SOLL,Arbeitsschritt,Arbeitsschrittbezeichnung,AFO_Start_SOLL,AFO_Ende_SOLL,AFO_Start_IST,AFO_Ende_IST,MaschinenID,Maschinenbezeichnung,Maschinenkapazität,AFO_delay_hours
0,95a859f51cf541e0b4aed3a38bb93065,1,Steuerventilmodul,2013-09-20,1,2014-01-01 11:32:00,1,Info,2014-01-01 07:00:00,2014-01-01 07:01:00,2014-01-01 07:00:00,2014-01-01 07:01:00,NaN,NaN,NaN,0.0
1,ed4e40cb93c04d0f9bcb8f7ecdc8752a,1,Steuerventilmodul,2013-11-09,1,2014-01-01 11:32:00,1,Info,2014-01-01 07:00:00,2014-01-01 07:01:00,2014-01-01 07:00:00,2014-01-01 07:01:00,NaN,NaN,NaN,0.0
2,ce233ad078b9429b8bd40f09100e8ee0,1,Steuerventilmodul,2013-12-30,1,2014-01-01 11:32:00,1,Info,2014-01-01 07:00:00,2014-01-01 07:01:00,2014-01-01 07:00:00,2014-01-01 07:01:00,NaN,NaN,NaN,0.0
3,c6b0430e1b7b4f328f0ac195c3070390,1,Steuerventilmodul,2013-10-05,1,2014-01-01 11:32:00,1,Info,2014-01-01 07:00:00,2014-01-01 07:01:00,2014-01-01 07:00:00,2014-01-01 07:01:00,NaN,NaN,NaN,0.0
4,5a5b4b41d6d246cfbe862018b557702b,1,Steuerventilmodul,2013-07-27,1,2014-01-01 11:32:00,1,Info,2014-01-01 07:00:00,2014-01-01 07:01:00,2014-01-01 07:00:00,2014-01-01 07:01:00,NaN,NaN,NaN,0.0


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1528238 entries, 0 to 1528237
Data columns (total 15 columns):
 #   Column                     Non-Null Count    Dtype         
---  ------                     --------------    -----         
 0   AuftragsID                 1528238 non-null  object        
 1   BauteilID                  1528238 non-null  int64         
 2   Bauteilbezeichnung         1528238 non-null  object        
 3   Auftragseingang            1528238 non-null  datetime64[ns]
 4   Priorität                  1528238 non-null  int64         
 5   Auftragsende_SOLL          1528238 non-null  datetime64[ns]
 6   Arbeitsschritt             1528238 non-null  int64         
 7   Arbeitsschrittbezeichnung  1528238 non-null  object        
 8   AFO_Start_SOLL             1528238 non-null  datetime64[ns]
 9   AFO_Ende_SOLL              1528238 non-null  datetime64[ns]
 10  AFO_Start_IST              1528238 non-null  datetime64[ns]
 11  AFO_Ende_IST               1528238 no

In [ ]:
data["AFO_delay_end_hours"] = (data["AFO_Ende_IST"] - data["AFO_Ende_SOLL"]).dt.total_seconds() / 3600
data["AFO_duration_hours"] = (data["AFO_Ende_IST"] - data["AFO_Start_IST"]).dt.total_seconds() / 3600
data["AFO_duration_soll_hours"] = (data["AFO_Ende_SOLL"] - data["AFO_Start_SOLL"]).dt.total_seconds() / 3600
data["AFO_effizienz"] = np.where(
    data["AFO_duration_hours"] > 0,
    data["AFO_duration_soll_hours"] / data["AFO_duration_hours"],
    np.nan
)
data["AFO_delay_start_hours"] = (data["AFO_Start_IST"] - data["AFO_Start_SOLL"]).dt.total_seconds() / 3600
# Zeitkomponenten aus Start-IST ziehen
data["jahr"] = data["AFO_Start_IST"].dt.year
data["monat"] = data["AFO_Start_IST"].dt.month
data["wochentag"] = data["AFO_Start_IST"].dt.dayofweek  # 0 = Montag
data["stunde"] = data["AFO_Start_IST"].dt.hour

data["is_high_capacity"] = data["Maschinenkapazität"] > data["Maschinenkapazität"].median()
data["is_high_priority"] = data["Priorität"] == 3

data["AFO_recovery_hours"] = data["AFO_delay_end_hours"] - data["AFO_delay_start_hours"]
data["is_last_afo"] = (
    data["AFO_Ende_IST"] == data.groupby("AuftragsID")["AFO_Ende_IST"].transform("max")
)

data["AFO_finish_gap_hours"] = np.where(
    data["is_last_afo"],
    (data["AFO_Ende_IST"] - data["Auftragsende_SOLL"]).dt.total_seconds() / 3600,
    np.nan
)
# Feature: Beginn außerhalb der Produktionszeit
data["is_outside_shift"] = (
    ~data["AFO_Start_IST"].dt.hour.between(7, 15)
    | (data["AFO_Start_IST"].dt.dayofweek > 4)  # 5 = Samstag, 6 = Sonntag
)

data["is_finished"] = data["AFO_Ende_IST"] < pd.Timestamp("2024-08-05")

data["is_late_and_outside_shift"] = (
    (data["AFO_delay_hours"] > 0) & data["is_outside_shift"]
)

data["is_legacy_order"] = data["AFO_Start_SOLL"] < "2014-01-01"

data["lead_time_hours"] = (
    (data["AFO_Start_SOLL"] - data["Auftragseingang"]).dt.total_seconds() / 3600
)

data["planned_start_to_end_gap_hours"] = (
    (data["Auftragsende_SOLL"] - data["AFO_Start_SOLL"]).dt.total_seconds() / 3600
)
data["start_ahead_or_late_hours"] = (
    (data["AFO_Start_IST"] - data["AFO_Start_SOLL"]).dt.total_seconds() / 3600
)

data["customer_due_gap_hours"] = (
    (data["AFO_Ende_IST"] - data["Auftragsende_SOLL"]).dt.total_seconds() / 3600
)

data["is_jit_success"] = data["customer_due_gap_hours"].between(-2, 2)

data["planned_duration_gap_hours"] = (
    data["AFO_duration_soll_hours"] - data["AFO_duration_hours"]
)


data["jit_utilization_ratio"] = (
    data["AFO_duration_hours"] / 
    ((data["Auftragsende_SOLL"] - data["AFO_Start_SOLL"]).dt.total_seconds() / 3600)
)

In [ ]:
# --- STANDARDARBEITSPLAN je Bauteil ermitteln ---

# Typische Reihenfolge der Arbeitsschritte je Bauteil
standard_arbeitsplan = (
    data.groupby(["Bauteilbezeichnung", "Arbeitsschrittbezeichnung"])
    .agg({
        "Arbeitsschritt": lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan,  # typische AFO-Nummer
        "Maschinenbezeichnung": lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan,  # häufigste Maschine
        "AFO_duration_hours": "median",  # typische Dauer
        "AFO_delay_end_hours": "median"  # typische Verspätung
    })
    .reset_index()
    .sort_values(["Bauteilbezeichnung", "Arbeitsschritt"])
)

# Das DataFrame kannst du später inspizieren:
# standard_arbeitsplan.head(10)

# Merge mit Standardarbeitsplan, um typische Maschine pro Schritt hinzuzufügen
data = data.merge(
    standard_arbeitsplan[["Bauteilbezeichnung", "Arbeitsschrittbezeichnung", "Maschinenbezeichnung"]],
    on=["Bauteilbezeichnung", "Arbeitsschrittbezeichnung"],
    how="left",
    suffixes=("", "_standard")
)

# Prüfen, ob aktuelle Maschine der typischen entspricht
data["is_standard_machine"] = data["Maschinenbezeichnung"] == data["Maschinenbezeichnung_standard"]

# Reihenfolge der AFOs je Auftrag
data["AFO_position"] = data.groupby("AuftragsID").cumcount() + 1

# Typische Reihenfolge je Bauteil (aus Standardarbeitsplan)
standard_order = (
    data.groupby(["Bauteilbezeichnung", "Arbeitsschrittbezeichnung"])["AFO_position"]
    .median()
    .reset_index()
    .rename(columns={"AFO_position": "AFO_standard_position"})
)

# Merge und Abweichung berechnen
data = data.merge(standard_order, on=["Bauteilbezeichnung", "Arbeitsschrittbezeichnung"], how="left")
data["AFO_sequence_deviation"] = data["AFO_position"] - data["AFO_standard_position"]


KeyError: "Column(s) ['AFO_delay_end_hours', 'AFO_duration_hours'] do not exist"

In [ ]:
from pandas.tseries.offsets import CustomBusinessHour
import pandas as pd
import numpy as np

# Definiere die Arbeitszeiten: Mo–Fr, 07:00–15:00 Uhr
custom_hours = CustomBusinessHour(start="07:00", end="15:00", weekmask="Mon Tue Wed Thu Fri")

# Funktion zur Berechnung der Bearbeitungszeit in Arbeitsstunden
def working_hours_diff(start, end):
    if pd.isna(start) or pd.isna(end):
        return np.nan
    return len(pd.date_range(start, end, freq=custom_hours))

# Beispiel für AFO-Dauer in Arbeitsstunden
data["AFO_duration_workhours"] = data.apply(
    lambda x: working_hours_diff(x["AFO_Start_IST"], x["AFO_Ende_IST"]),
    axis=1
)

data["AFO_duration_workhours"] = data.apply(
    lambda x: working_hours_diff(x["AFO_Start_IST"], x["AFO_Ende_IST"]),
    axis=1
)

data["AFO_delay_end_workhours"] = data.apply(
    lambda x: working_hours_diff(x["AFO_Ende_SOLL"], x["AFO_Ende_IST"]),
    axis=1
)


In [ ]:
data["AFO_delay_hours"].describe(percentiles=[.05, .25, .5, .75, .95])


count    1.528238e+06
mean     1.252858e+03
std      2.559999e+03
min     -3.106833e+02
5%      -1.710000e+01
25%      0.000000e+00
50%      2.300000e+00
75%      9.180000e+01
95%      7.240406e+03
max      1.005082e+04
Name: AFO_delay_hours, dtype: float64